In [21]:
# %% md
import torch
from utils import accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.retiarii.oneshot.pytorch import DartsTrainer, EnasTrainer
import itertools
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn
from collections import OrderedDict

In [22]:
def reward_accuracy(output, target, topk=(1,)):
    batch_size = target.size(0)
    _, predicted = torch.max(output.data, 1)
    return (predicted == target).sum().item() / batch_size

In [1]:
# Retiarii Example - One-shot NAS

## Step 1: Express the Model Space

### Step 1.1: Define the Base Model

class CIFAR_17(nn.Module):
    '''
    BaseModel which has 3 CNN layers and 2 FC layers
    '''

    def __init__(self, head_size=10):
        super(CIFAR_17, self).__init__()

        self.body = nn.Sequential(OrderedDict([
            ('cnn1', nn.Sequential(OrderedDict([
                ('conv', nn.Conv2d(3, 8, 3, 1, 1)),
                ('batchnorm', nn.BatchNorm2d(8)),
                ('relu', nn.ReLU(inplace=True)),
                ('pool', nn.MaxPool2d(2))
            ]))),
            ('cnn2', nn.Sequential(OrderedDict([
                ('conv', nn.Conv2d(8, 8, 3, 1, 1)),
                ('batchnorm', nn.BatchNorm2d(8)),
                ('relu', nn.ReLU(inplace=True)),
                ('pool', nn.MaxPool2d(2))
            ]))),
            ('cnn3', nn.Sequential(OrderedDict([
                ('conv', nn.Conv2d(8, 8, 3, 1, 1)),
                ('batchnorm', nn.BatchNorm2d(8)),
                ('relu', nn.ReLU(inplace=True)),
                ('pool', nn.MaxPool2d(2)),
            ])))
        ]))

        self.head = nn.Sequential(OrderedDict([
            ('dense', nn.Sequential(OrderedDict([
                ('fc1', nn.Conv2d(8 * 4 * 4, 32, kernel_size=1, bias=True)),  # implement dense layer in CNN way
                ('relu', nn.ReLU(inplace=True)),
                ('fc2', nn.Conv2d(32, head_size, kernel_size=1, bias=True)),
            ])))
        ]))

    def features(self, x):
        feat = self.body(x)
        feat = x.view(x.shape[0], -1)
        return feat

    def forward(self, x):
        x = self.body(x)
        x = x.view(x.shape[0], -1, 1, 1)  # flatten
        x = self.head(x)
        x = x.view(x.shape[0], -1)
        return x

model = CIFAR_17()

In [2]:
### Step 1.2: Define the Model Mutations


import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn


class Net(nn.Module):
    def __init__(self, head_size=10, lower_range=8, upper_range=16):
        super(Net, self).__init__()
        self.head_size = head_size
        self.lower_range = lower_range
        self.upper_range = upper_range
        choice_dict = self._get_mutator()
        self.net = nn.LayerChoice(choice_dict)

    def _get_mutator(self):
        ## this is supposed to be slooow 
        layer_choices = []
        a = [range(self.lower_range, self.upper_range+1),
             range(self.lower_range, self.upper_range+1),
             range(self.lower_range, self.upper_range+1)]

        for comb in list(itertools.product(*a)):
            i, j, k = comb
            layer_choices.append(
                nn.Sequential(
                        nn.Conv2d(3, i, 3, 1, 1),
                        nn.BatchNorm2d(i),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2),

                        nn.Conv2d(i, j, 3, 1, 1),
                        nn.BatchNorm2d(j),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2),

                        nn.Conv2d(j, k, 3, 1, 1),
                        nn.BatchNorm2d(k),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2),

                        nn.Flatten(),
                        nn.Linear(k * 4 * 4, 32),
                        nn.ReLU(inplace=True),
                        nn.Linear(32, self.head_size),
                    )
            )
        return layer_choices

    def forward(self, x):
        out = self.net(x)
        return out


model = Net()
model.forward(torch.rand(1, 3, 32, 32))

/home/l/liny/anaconda3/envs/nni/lib/python3.7/site-packages/nni/retiarii/nn/pytorch/api.py:135: UserWarning: You should not run forward of this module directly.
  warnings.warn('You should not run forward of this module directly.')


tensor([[[[0.9952, 0.9952, 0.7538,  ..., 0.0784, 0.2492, 0.8992],
          [0.2803, 0.4774, 0.6050,  ..., 0.5419, 0.2649, 0.8008],
          [0.5444, 0.0655, 0.2073,  ..., 0.4301, 0.3077, 0.3610],
          ...,
          [0.8340, 0.1187, 0.4737,  ..., 0.9927, 0.7127, 0.7953],
          [0.4574, 0.8672, 0.4332,  ..., 0.9585, 0.2413, 0.0166],
          [0.5465, 0.4179, 0.2710,  ..., 0.2023, 0.2166, 0.6758]],

         [[0.8038, 0.8533, 0.1094,  ..., 0.6518, 0.8641, 0.4620],
          [0.6234, 0.8136, 0.4608,  ..., 0.8882, 0.2297, 0.3186],
          [0.0039, 0.7103, 0.0429,  ..., 0.5362, 0.3790, 0.7737],
          ...,
          [0.8671, 0.5567, 0.3177,  ..., 0.1622, 0.6184, 0.5844],
          [0.6359, 0.2116, 0.0425,  ..., 0.9059, 0.7966, 0.0073],
          [0.9150, 0.9033, 0.6138,  ..., 0.2719, 0.9967, 0.4664]],

         [[0.1611, 0.3259, 0.5206,  ..., 0.9684, 0.1247, 0.4120],
          [0.7703, 0.5999, 0.8896,  ..., 0.4230, 0.1522, 0.8901],
          [0.1185, 0.5245, 0.9828,  ..., 0

In [3]:
## Step 2: Explore the Model Space

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.001, 
                            momentum=0.9)

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                     (0.2023, 0.1994, 0.2010)),
                               ])

train_dataset = CIFAR10(root="./data",
                        train=True,
                        download=True,
                        transform=transform)


Files already downloaded and verified


## DARTS or ENAS

In [24]:
trainer = EnasTrainer(
        model=model, 
        loss=criterion,
        metrics=lambda output, target: accuracy(output, target), 
        optimizer=optimizer,
        num_epochs=2, 
        dataset=train_dataset,
        batch_size=8, 
        log_frequency=10,
        reward_function=reward_accuracy,
#         device=torch.device("cpu")
    
)
        

trainer.fit()

[2021-06-13 15:23:19] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1/3125]  acc1 0.375000 (0.375000)  loss 2.282778 (2.282778)
[2021-06-13 15:23:44] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [11/3125]  acc1 0.250000 (0.272727)  loss 2.296400 (2.284443)
[2021-06-13 15:24:08] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [21/3125]  acc1 0.000000 (0.244048)  loss 2.296896 (2.284458)
[2021-06-13 15:24:32] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [31/3125]  acc1 0.125000 (0.229839)  loss 2.296020 (2.285962)
[2021-06-13 15:24:57] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [41/3125]  acc1 0.125000 (0.228659)  loss 2.282803 (2.286016)
[2021-06-13 15:25:21] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [51/3125]  acc1 0.500000 (0.240196)  loss 2.273415 (2.285844)
[2021-06-13 15:25:45] INFO (nni.retiarii.onesho

[2021-06-13 15:44:20] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [521/3125]  acc1 0.375000 (0.246401)  loss 2.271954 (2.285692)
[2021-06-13 15:44:45] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [531/3125]  acc1 0.125000 (0.245998)  loss 2.277123 (2.285604)
[2021-06-13 15:45:09] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [541/3125]  acc1 0.125000 (0.245610)  loss 2.292711 (2.285618)
[2021-06-13 15:45:33] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [551/3125]  acc1 0.125000 (0.246597)  loss 2.284531 (2.285526)
[2021-06-13 15:45:57] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [561/3125]  acc1 0.125000 (0.246881)  loss 2.304264 (2.285581)
[2021-06-13 15:46:21] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [571/3125]  acc1 0.250000 (0.246935)  loss 2.305649 (2.285599)
[2021-06-13 15:46:46] INFO (nni.retiarii

[2021-06-13 16:05:21] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1041/3125]  acc1 0.125000 (0.248439)  loss 2.295474 (2.285356)
[2021-06-13 16:05:45] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1051/3125]  acc1 0.125000 (0.248692)  loss 2.301641 (2.285314)
[2021-06-13 16:06:10] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1061/3125]  acc1 0.250000 (0.249411)  loss 2.277019 (2.285259)
[2021-06-13 16:06:34] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1071/3125]  acc1 0.375000 (0.249883)  loss 2.283672 (2.285248)
[2021-06-13 16:06:58] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1081/3125]  acc1 0.250000 (0.249537)  loss 2.285428 (2.285220)
[2021-06-13 16:07:22] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1091/3125]  acc1 0.125000 (0.248969)  loss 2.302309 (2.285231)
[2021-06-13 16:07:47] INFO (nni.re

[2021-06-13 16:27:59] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1601/3125]  acc1 0.500000 (0.248985)  loss 2.286031 (2.284901)
[2021-06-13 16:28:23] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1611/3125]  acc1 0.250000 (0.248836)  loss 2.272707 (2.284902)
[2021-06-13 16:28:48] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1621/3125]  acc1 0.375000 (0.249152)  loss 2.271130 (2.284881)
[2021-06-13 16:29:12] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1631/3125]  acc1 0.375000 (0.249157)  loss 2.271302 (2.284855)
[2021-06-13 16:29:36] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1641/3125]  acc1 0.000000 (0.248934)  loss 2.286549 (2.284845)
[2021-06-13 16:30:00] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [1651/3125]  acc1 0.250000 (0.248864)  loss 2.281196 (2.284835)
[2021-06-13 16:30:25] INFO (nni.re

[2021-06-13 16:48:37] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2111/3125]  acc1 0.000000 (0.252191)  loss 2.277981 (2.284157)
[2021-06-13 16:49:01] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2121/3125]  acc1 0.250000 (0.252122)  loss 2.291888 (2.284133)
[2021-06-13 16:49:26] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2131/3125]  acc1 0.125000 (0.252229)  loss 2.296679 (2.284123)
[2021-06-13 16:49:50] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2141/3125]  acc1 0.250000 (0.252394)  loss 2.282331 (2.284111)
[2021-06-13 16:50:14] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2151/3125]  acc1 0.500000 (0.252731)  loss 2.276115 (2.284076)
[2021-06-13 16:50:39] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2161/3125]  acc1 0.375000 (0.253008)  loss 2.260893 (2.284035)
[2021-06-13 16:51:03] INFO (nni.re

[2021-06-13 17:09:14] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2621/3125]  acc1 0.000000 (0.251145)  loss 2.300082 (2.283673)
[2021-06-13 17:09:38] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2631/3125]  acc1 0.125000 (0.250950)  loss 2.281259 (2.283670)
[2021-06-13 17:10:03] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2641/3125]  acc1 0.000000 (0.251041)  loss 2.293207 (2.283648)
[2021-06-13 17:10:27] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2651/3125]  acc1 0.250000 (0.251084)  loss 2.281216 (2.283635)
[2021-06-13 17:10:51] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2661/3125]  acc1 0.500000 (0.251362)  loss 2.266122 (2.283612)
[2021-06-13 17:11:16] INFO (nni.retiarii.oneshot.pytorch.enas/MainThread) Model Epoch [1/2] Step [2671/3125]  acc1 0.250000 (0.251685)  loss 2.285561 (2.283588)
[2021-06-13 17:11:40] INFO (nni.re

AttributeError: 'int' object has no attribute 'item'

In [26]:
# Similarly, the optimal structure found can be exported.
print('Final architecture:', trainer.export())

Final architecture: {}


In [17]:
trainer.nas_modules[0][1].op_choices['278']

Sequential(
  (0): Conv2d(3, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(11, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(11, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=256, out_features=32, bias=True)
  (14): ReLU(inplace=True)
  (15): Linear(in_features=32, out_features=10, bias=Tr

## TPE / RegularizedEvolution/ RS / GS

In [ ]:
import nni.retiarii.strategy as strategy
simple_strategy = strategy.TPEStrategy() # choice: Random, GridSearch, RegularizedEvolution, TPEStrategy


In [ ]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.retiarii import serialize
import nni.retiarii.evaluator.pytorch.lightning as pl

transform = transforms.Compose([transforms.ToTensor(), 
                                
                                transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                     (0.2023, 0.1994, 0.2010)),
                               ])


train_dataset = serialize(CIFAR10, root="./data", train=True, download=True, transform=transform)
test_dataset = serialize(CIFAR10, root="./data", train=False, download=True, transform=transform)

trainer = pl.Classification(train_dataloader=pl.DataLoader(train_dataset, batch_size=64),
                            val_dataloaders=pl.DataLoader(test_dataset, batch_size=64),
                            max_epochs=2, gpus=[0])


In [ ]:
from nni.retiarii.experiment.pytorch import RetiariiExeConfig, RetiariiExperiment

exp = RetiariiExperiment(model, trainer, [], simple_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'Retiarii example'
exp_config.trial_concurrency = 2
exp_config.max_trial_number = 10
exp_config.trial_gpu_number = 0
exp_config.max_experiment_duration = '5m'
exp_config.training_service.use_active_gpu = True


In [ ]:
exp.run(exp_config, 8745)


In [ ]:
print('Final model:')
for model_code in exp.export_top_models():
    print(model_code)
